## MOVIE BUDGETS

In [87]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os

%matplotlib inline

In [88]:
# load the data
movie_budgets_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz', encoding='utf-8')
movie_budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


## Data Preparation

### Data Cleaning

1. **Renaming Columns**:
   - The column `movie` is renamed to `movie_name` for better clarity and consistency with other datasets.

2. **Converting `release_date` to Datetime Format**:
   - The `release_date` column is converted to a datetime format using `pd.to_datetime()`. This ensures that the column can be used for date-based operations, such as extracting the year or filtering by date.

3. **Standardizing the `movie_name` Column**:
   - The `movie_name` column is cleaned by:
     - Stripping leading and trailing spaces using `.str.strip()`.
     - Converting all text to lowercase using `.str.lower()`.
   - This standardization ensures consistency in movie names, making it easier to identify duplicates or merge with other datasets.

In [89]:
# Rename 'movie' column to 'movie_name'
movie_budgets_df.rename(columns={'movie': 'movie_name'}, inplace=True)

# Convert 'release_date' to datetime format
movie_budgets_df['release_date'] = pd.to_datetime(movie_budgets_df['release_date'])

# Standardize the 'movie_name' column
movie_budgets_df['movie_name'] = movie_budgets_df['movie_name'].str.strip().str.lower()

# Display the updated column names
print(movie_budgets_df.columns)
# Display the updated DataFrame
print(movie_budgets_df.dtypes)

Index(['id', 'release_date', 'movie_name', 'production_budget',
       'domestic_gross', 'worldwide_gross'],
      dtype='object')
id                            int64
release_date         datetime64[ns]
movie_name                   object
production_budget            object
domestic_gross               object
worldwide_gross              object
dtype: object


In [90]:
# Check for missing and null values
movie_budgets_df.isna().sum()

id                   0
release_date         0
movie_name           0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

#### Data Transformation: 

Converting data into a more useful format 
1. **Extract Year**: The `dt.year` attribute extracts the year from the `release_date` column.
2. **Identify Duplicates**: The `duplicated()` method identifies duplicate `movie_name` entries.

    - It identifies rows with duplicate movie names and then modifies the 'movie_name' for these duplicates by appending their release year in parentheses. 
    - This is to distinguish between movies with the same title but different release dates, creating more unique identifiers.
    
3. **Update Names**: For duplicates, the `movie_name` is updated by appending the release year in parentheses.

This ensures that duplicate movie names are uniquely identified by their release year.

In [91]:
# check for duplicates using movie_name and release_date
duplicates = movie_budgets_df[movie_budgets_df.duplicated(subset=['movie_name'], keep=False)]
# Display duplicates sorted by movie_name in ascending order
duplicates = duplicates.sort_values(by='movie_name', ascending=True)
print(duplicates.shape)
duplicates.head(10)

(165, 6)


,id,release_date,movie_name,production_budget,domestic_gross,worldwide_gross
4270,71,1954-12-23,"20,000 leagues under the sea","$5,000,000","$28,200,000","$28,200,000"
5614,15,1916-12-24,"20,000 leagues under the sea","$200,000","$8,000,000","$8,000,000"
1648,49,2010-04-30,a nightmare on elm street,"$35,000,000","$63,075,011","$117,729,621"
5016,17,1984-11-09,a nightmare on elm street,"$1,800,000","$25,504,513","$25,504,513"
2032,33,1992-11-11,aladdin,"$28,000,000","$217,350,219","$504,050,219"
80,81,2019-05-24,aladdin,"$182,000,000","$246,734,314","$619,234,314"
50,51,2010-03-05,alice in wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
4759,60,1951-07-28,alice in wonderland,"$3,000,000",$0,$0
4120,21,1956-10-17,around the world in 80 days,"$6,000,000","$42,000,000","$42,000,000"
340,41,2004-06-16,around the world in 80 days,"$110,000,000","$24,004,159","$72,004,159"


In [92]:
# Extract the year from the 'release_date' column
movie_budgets_df['release_year'] = movie_budgets_df['release_date'].dt.year

# Check for duplicates in 'movie_name'
duplicates = movie_budgets_df[movie_budgets_df.duplicated(subset=['movie_name'], keep=False)]

# Update 'movie_name' for duplicates by appending the release year
movie_budgets_df.loc[duplicates.index, 'movie_name'] = (
    movie_budgets_df.loc[duplicates.index, 'movie_name'] + 
    " (" + movie_budgets_df.loc[duplicates.index, 'release_year'].astype(str) + ")"
)

# Display the updated DataFrame
movie_budgets_df[['movie_name', 'release_year']].head()

,movie_name,release_year
0,avatar,2009
1,pirates of the caribbean: on stranger tides,2011
2,dark phoenix,2019
3,avengers: age of ultron,2015
4,star wars ep. viii: the last jedi,2017


In [93]:
# Filter movie names containing the word 'alice in wonderland'
avenger_movies = movie_budgets_df[movie_budgets_df['movie_name'].str.contains('alice in wonderland', case=False, na=False)]

# Display the filtered DataFrame
print(f"Number of movies with 'alice in wonderland' in the name: {avenger_movies.shape[0]}")
avenger_movies[['movie_name', 'release_year']].head()

Number of movies with 'alice in wonderland' in the name: 2


,movie_name,release_year
50,alice in wonderland (2010),2010
4759,alice in wonderland (1951),1951


This focuses on **cleaning and converting financial columns** in the `movie_budgets_df` DataFrame.

For each of the 'production\_budget', 'domestic\_gross', and 'worldwide\_gross' columns, it performs the following actions:

1.  **Removes special characters:** It converts the column to string format, then uses regular expressions to remove dollar signs (\$) and commas (,).

2.  **Converts to numeric:** It convertS the cleaned strings to a numeric format. The `errors='coerce'` argument ensures that any values that cannot be converted to numbers will be replaced with `NaN` (Not a Number).

3.  **Converts to integer:** Finally, it converts the numeric data type to `Int64`, which is a pandas integer type that can handle missing values (represented by `NaN`).

**This cleans the budget and gross revenue columns by removing formatting characters and converting them into a usable integer format that can handle potential missing data.**

This is crucial for performing calculations and analysis on the financial aspects of the movies.

In [94]:
# Remove special characters ('$' and ',') and convert to Int64
movie_budgets_df['production_budget'] = pd.to_numeric(
    movie_budgets_df['production_budget'].astype(str).str.replace('[\$,]', '', regex=True), errors='coerce'
).astype('Int64')

movie_budgets_df['domestic_gross'] = pd.to_numeric(
    movie_budgets_df['domestic_gross'].astype(str).str.replace('[\$,]', '', regex=True), errors='coerce'
).astype('Int64')

movie_budgets_df['worldwide_gross'] = pd.to_numeric(
    movie_budgets_df['worldwide_gross'].astype(str).str.replace('[\$,]', '', regex=True), errors='coerce'
).astype('Int64')

# Display the updated DataFrame
print(movie_budgets_df.dtypes)
movie_budgets_df.head()

id                            int64
release_date         datetime64[ns]
movie_name                   object
production_budget             Int64
domestic_gross                Int64
worldwide_gross               Int64
release_year                  int64
dtype: object


,id,release_date,movie_name,production_budget,domestic_gross,worldwide_gross,release_year
0,1,2009-12-18,avatar,425000000,760507625,2776345279,2009
1,2,2011-05-20,pirates of the caribbean: on stranger tides,410600000,241063875,1045663875,2011
2,3,2019-06-07,dark phoenix,350000000,42762350,149762350,2019
3,4,2015-05-01,avengers: age of ultron,330600000,459005868,1403013963,2015
4,5,2017-12-15,star wars ep. viii: the last jedi,317000000,620181382,1316721747,2017


**Identifying and removing columns from a DataFrame that are not necessary or useful for analysis.**

This process streamlines the dataset by:

* **Reducing dimensionality:** Making the DataFrame easier to work with and potentially improving the performance of analytical models.
* **Removing noise:** Eliminating columns that don't contribute meaningful information and could even introduce confusion or bias.
* **Focusing on relevant data:** Ensuring that subsequent analysis is concentrated on the variables that truly matter for the research questions or objectives.

In [95]:
# Drop rows where 'production_budget' is less than 0
movie_budgets_df = movie_budgets_df[movie_budgets_df['production_budget'] >= 0]

# Display the updated DataFrame
print(f"Shape after dropping rows with production_budget < 0: {movie_budgets_df.shape}")

Shape after dropping rows with production_budget < 0: (5782, 7)


In [96]:
# Drop rows where both 'domestic_gross' and 'worldwide_gross' are 0
movie_budgets_df = movie_budgets_df[
    ~((movie_budgets_df['domestic_gross'] == 0) & (movie_budgets_df['worldwide_gross'] == 0))
]

# Display the updated DataFrame shape
print(f"Shape after dropping rows with zero gross: {movie_budgets_df.shape}")

Shape after dropping rows with zero gross: (5415, 7)


In [97]:
output_folder = './extractedData'
os.makedirs(output_folder, exist_ok=True)

# Save movie_basics_df to a CSV file
movie_budgets_df.to_csv(f'{output_folder}/cleaned_movie_budgets_df.csv', index=False)

print("DataFrames have been successfully saved to the 'extracted' folder.")

DataFrames have been successfully saved to the 'extracted' folder.


## COMBINE SQL DATA WITH MOVIE BUDGETS

To combine movies_df.csv and movie_budgets_df, you can use the following code:

In [98]:
# Load the cleaned movies_df.csv
movies_df = pd.read_csv('./cleaned_data/movies_df.csv')
movie_budgets_df = pd.read_csv('./extractedData/cleaned_movie_budgets_df.csv')

movies_df.info()
print('--------' * 20)
movie_budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10010 entries, 0 to 10009
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         10010 non-null  object 
 1   primary_title    10010 non-null  object 
 2   start_year       10010 non-null  int64  
 3   runtime_minutes  10010 non-null  int64  
 4   genres           10010 non-null  object 
 5   averagerating    10010 non-null  float64
 6   numvotes         10010 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 547.5+ KB
----------------------------------------------------------------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5415 entries, 0 to 5414
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5415 non-null   int64 
 1   r

**Renaming columns and converting the release year to a datetime object** 

This standardizes the column names for 'movie\_name' and 'release\_year'

In [99]:
# change primary_title to movie_name, start_year to release_year
movies_df.rename(columns={'primary_title': 'movie_name', 'start_year': 'release_year'}, inplace=True)


This combines movie details (from `movies_df`) with budget and revenue information (from `movie_budgets_df`) into a single DataFrame for further analysis. It ensures that only movies present in both datasets are included.
1. **Merging DataFrames**:
   - The `movies_df` and `movie_budgets_df` DataFrames are merged on the `movie_name` column.
   - An **inner join** is used, meaning only rows with matching `movie_name` values in both DataFrames are included in the resulting `combined_data` DataFrame.

2. **Adding Suffixes**:
   - Suffixes `('_movies', '_budgets')` are added to distinguish columns with the same name in both DataFrames (e.g., `release_year_movies` and `release_year_budgets`).

3. **Displaying Results**:
   - The shape of the merged DataFrame (`combined_data`) is printed to show the number of rows and columns.
   - The first few rows of the merged DataFrame are displayed using `.head()` to verify the merge.

In [100]:
# Merge movies_df and movie_budgets_df on 'movie_name'
combined_data = pd.merge(
    movies_df,
    movie_budgets_df,
    on='movie_name',
    how='inner',  # Use 'inner' to keep only matching rows
    suffixes=('_movies', '_budgets')  # Add suffixes to distinguish columns
)

# Display the shape and the first few rows of the combined DataFrame
print(f"Shape of the combined DataFrame: {combined_data.shape}")
combined_data.head()

Shape of the combined DataFrame: (1426, 13)


,movie_id,movie_name,release_year_movies,runtime_minutes,genres,averagerating,numvotes,id,release_date,production_budget,domestic_gross,worldwide_gross,release_year_budgets
0,tt0249516,foodfight!,2012,91,"action,animation,comedy",1.9,8248,26,2012-12-31,45000000,0,73706,2012
1,tt0359950,the secret life of walter mitty,2013,114,"adventure,comedy,drama",7.3,275300,37,2013-12-25,91000000,58236838,187861183,2013
2,tt0365907,a walk among the tombstones,2014,114,"action,crime,drama",6.5,105116,67,2014-09-19,28000000,26017685,62108587,2014
3,tt0369610,jurassic world,2015,124,"action,adventure,sci-fi",7.0,539338,34,2015-06-12,215000000,652270625,1648854864,2015
4,tt0376136,the rum diary,2011,119,"comedy,drama",6.2,94787,16,2011-10-28,45000000,13109815,21544732,2011


To check if the release_year_movies and release_year_budgets columns do not match in the combined_data DataFrame

1. **Filter Rows with Mismatched Years**:
   - The condition `combined_data['release_year_movies'] != combined_data['release_year_budgets']` identifies rows where the release years in the two columns do not match.

2. **Count Mismatched Rows**:
   - `mismatched_years.shape[0]` gives the total number of rows with mismatched release years.

3. **Display Results**:
   - The `head()` method displays the first few rows of the mismatched data, showing the `movie_name`, `release_year_movies`, and `release_year_budgets` columns for inspection.

In [103]:
# Filter rows where the absolute difference between release_year_movies and release_year_budgets is greater than 10
large_mismatched_years = combined_data[
    (combined_data['release_year_movies'] - combined_data['release_year_budgets']).abs() > 10
]

# Display the count and the mismatched rows
print(f"Number of rows with a year difference greater than 10: {large_mismatched_years.shape[0]}")
large_mismatched_years[['movie_name', 'release_year_movies', 'release_year_budgets']].head()

Number of rows with a year difference greater than 10: 45


,movie_name,release_year_movies,release_year_budgets
8,action jackson,2014,1988
106,fair game,2010,1995
404,no man's land,2013,2001
433,vampires,2010,1998
464,playing for keeps,2012,1986


In [104]:
# Shape of the combined DataFrame before dropping rows
print(f"Shape of the combined DataFrame before dropping rows: {combined_data.shape}")   

# DROP ROWS WHERE THE YEAR DIFFERENCE IS GREATER THAN 10
combined_data = combined_data[
    (combined_data['release_year_movies'] - combined_data['release_year_budgets']).abs() <= 10
]

print(f"Shape of the combined DataFrame after dropping rows: {combined_data.shape}")

Shape of the combined DataFrame before dropping rows: (1426, 13)
Shape of the combined DataFrame after dropping rows: (1381, 13)


In [105]:
# We check if the data types are consistent across the combined DataFrame
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1381 entries, 0 to 1425
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              1381 non-null   object 
 1   movie_name            1381 non-null   object 
 2   release_year_movies   1381 non-null   int64  
 3   runtime_minutes       1381 non-null   int64  
 4   genres                1381 non-null   object 
 5   averagerating         1381 non-null   float64
 6   numvotes              1381 non-null   int64  
 7   id                    1381 non-null   int64  
 8   release_date          1381 non-null   object 
 9   production_budget     1381 non-null   int64  
 10  domestic_gross        1381 non-null   int64  
 11  worldwide_gross       1381 non-null   int64  
 12  release_year_budgets  1381 non-null   int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 151.0+ KB


In [106]:
# check for duplicates in the combined DataFrame based on 'movie_name'
duplicates_combined = combined_data[combined_data.duplicated(subset=['movie_name'], keep=False)]
duplicates_combined

,movie_id,movie_name,release_year_movies,runtime_minutes,genres,averagerating,numvotes,id,release_date,production_budget,domestic_gross,worldwide_gross,release_year_budgets


In [107]:
# drop irelevant columns
drop_columns = [ 'id', 'release_year_budgets']
combined_data.drop(columns=drop_columns, inplace=True)

# Rename release_year_movies to release_year
combined_data.rename(columns={'release_year_movies': 'release_year'}, inplace=True)


In [108]:
# This will display the data types of the remaining columns in the combined DataFrame
combined_data.dtypes

movie_id              object
movie_name            object
release_year           int64
runtime_minutes        int64
genres                object
averagerating        float64
numvotes               int64
release_date          object
production_budget      int64
domestic_gross         int64
worldwide_gross        int64
dtype: object

In [109]:
# Convert 'release_date' to datetime format
combined_data['release_date'] = pd.to_datetime(combined_data['release_date'])

# Display the updated DataFrame to confirm the change
print(combined_data.dtypes)

movie_id                     object
movie_name                   object
release_year                  int64
runtime_minutes               int64
genres                       object
averagerating               float64
numvotes                      int64
release_date         datetime64[ns]
production_budget             int64
domestic_gross                int64
worldwide_gross               int64
dtype: object


In [110]:
output_folder = './cleaned_data'
os.makedirs(output_folder, exist_ok=True)

# Save movie_basics_df to a CSV file
combined_data.to_csv(f'{output_folder}/movie_budgets.csv', index=False)

print("DataFrames have been successfully saved to the 'extracted' folder.")

DataFrames have been successfully saved to the 'extracted' folder.


### Conclusion:

The `movie_budgets_df` dataset is successfully cleaned and prepared for analysis by:
 - renaming columns, 
 - converting `release_date` to datetime format, 
 - standardizing `movie_name`
 - cleaning financial columns. 
 
It is then merged with `movies_df` to create a combined dataset, ensuring only matching movies were included. 
This combined dataset is now ready for further analysis, with mismatched release years identified and irrelevant columns removed for clarity and focus.